In [1]:
import pickle
import pandas as pd


In [2]:
df=pd.read_csv('data.csv')
df

,text,label
0,d,1
1,An Album I can't get enough of!: When I herd t...,0
2,This was a waste of time.: Let's keep this sim...,0
3,Perfect gift for the family: Purchased iwth Am...,0
4,cddddd,1
...,...,...
5132,gfnd htdgfvcx,1
5133,Unbelievable: I'll have to reserve a higher ra...,0
5134,Wouldn't recommend: The sizing of this costume...,0
5135,jnkfvjkf,1


In [4]:
df.isna().sum()

text     0
label    0
dtype: int64

In [5]:
df.head(10)

,text,label
0,d,1
1,An Album I can't get enough of!: When I herd t...,0
2,This was a waste of time.: Let's keep this sim...,0
3,Perfect gift for the family: Purchased iwth Am...,0
4,cddddd,1
5,Fhhbc,1
6,rbgyuv6,1
7,asadasd,1
8,fhfghfghfg,1
9,One of the best: My wife purchased this game f...,0


#### Preprocessing If Required

In [ ]:
# import re
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# nltk.download('punkt')
# nltk.download('stopwords')

# def preprocess_text(text):
#     # Convert to lowercase
#     text = text.lower()
    
#     # Remove punctuation
#     text = re.sub(r'[^\w\s]', '', text)
    
# # #     # Tokenize the text
#     tokens = word_tokenize(text)
# # #     # Remove stopwords
# # #     stop_words = set(stopwords.words('english'))
# # #     tokens = [word for word in tokens if word not in stop_words]
    
#     # Join tokens back to a clean text
#     clean_text = ' '.join(tokens)
    
#     return text

# df['text_cleaned'] = [preprocess_text(sample) for sample in df['text']]

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
X=df['text']
y=df['label']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape,y_train.shape

((4109,), (4109,))

## Saving Test Data 

In [57]:
test=pd.DataFrame(X_test)
test['label']=y_test.astype(int)
test.to_csv('test.csv',index=False)

In [31]:
# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train)

with open('vectorizer.pk', 'wb') as gibd:
    pickle.dump(vectorizer, gibd)
# with open('vectorizer.pk', 'rb') as gibd_r:
#     vectorizer_load=pickle.load(gibd_r)

In [32]:
X_train_vec

<4109x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 103753 stored elements in Compressed Sparse Row format>

In [21]:

wordVector = CountVectorizer()

#Build Vocab
finalWordVocab = wordVector.fit(X_train)

In [27]:
bagOfWords = finalWordVocab.transform(X_train)

In [43]:
bagOfWords_test = finalWordVocab.transform(X_test)

In [28]:
bagOfWords

<4109x16131 sparse matrix of type '<class 'numpy.int64'>'
	with 109884 stored elements in Compressed Sparse Row format>

In [34]:
from sklearn.feature_extraction.text import TfidfTransformer
tfIdfObject = TfidfTransformer().fit(bagOfWords)

#Transform data (Calc Weights )
finalFeature = tfIdfObject.transform(bagOfWords)

In [45]:
from sklearn.feature_extraction.text import TfidfTransformer
# tfIdfObject = TfidfTransformer().fit(bagOfWords)

#Transform data (Calc Weights )
finalFeature_test = tfIdfObject.transform(bagOfWords_test)

In [35]:
finalFeature

<4109x16131 sparse matrix of type '<class 'numpy.float64'>'
	with 109884 stored elements in Compressed Sparse Row format>

In [24]:
# finalWordVocab.vocabulary_

finalFeature### Naive Bayes Classifier

In [37]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(finalFeature.toarray(), y_train)

GaussianNB()

In [38]:
with open('gnb_model.pkl', 'wb') as gnb_model:
    pickle.dump(gnb, gnb_model)

In [39]:
with open('gnb_model.pkl', 'rb') as gnb_saved_model:
    model=pickle.load(gnb_saved_model)

In [40]:
X_test_vec = vectorizer.transform(X_test)

In [41]:
y_test.shape

(1028,)

In [46]:
y_pred = model.predict(finalFeature_test.toarray())
y_pred

array([1, 1, 0, ..., 1, 0, 0], dtype=int64)

In [47]:
y_pred.shape

(1028,)

In [48]:
# Create confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Confusion Matrix:
 [[529   0]
 [ 13 486]]
Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       529
           1       1.00      0.97      0.99       499

    accuracy                           0.99      1028
   macro avg       0.99      0.99      0.99      1028
weighted avg       0.99      0.99      0.99      1028



### Random Forest Classifier

In [25]:
# Train the model
rf_model = RandomForestClassifier()
rf_model.fit(X_train_vec, y_train)



RandomForestClassifier()

In [26]:
with open('rf_model.pkl', 'wb') as rf_m:
    pickle.dump(rf_model, rf_m)

In [27]:
with open('rf_model.pkl', 'rb') as rf_ml:
    model_rf=pickle.load(rf_ml)

In [28]:
# Predict on test data
y_pred = model_rf.predict(X_test_vec)

# Create confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Confusion Matrix:
 [[526   3]
 [  1 498]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00       529
           1       0.99      1.00      1.00       499

    accuracy                           1.00      1028
   macro avg       1.00      1.00      1.00      1028
weighted avg       1.00      1.00      1.00      1028



In [2]:
# !pip freeze